# Arduino

Let's say we want to train a model for our Arduino (We are using Arduino Nano 33 BLE Sense Rev2 for this example).

First we would need some data. There are two fundemental ways to stream data from the arduino to our computer.

    1. Using the serial port
    2. Using the BLE

# Is the BLESignalSource only Arduino compatible ?

Let's first look at the serial port.

In [1]:
import os

os.chdir("../")

Let's stream mic samples from the arduino board. 

We use the PDM library to sample our mic and then stream 256 samples (512 bytes) over the serial port at a time

Then we need to create a SignalSource to receive these samples. Let's write this as a Sampler.

A Sampler needs 3 methods
    
    1. start
    2. stop
    3. read

start should open the serial port and start processing the samples in a seperate thread.

stop should stop the processing, close the serial port and join the thread

and read returns all the samples read so far.



In [ ]:
from genki_signals.signal_sources.base import SamplerBase
from genki_signals.buffers import DataBuffer
import serial
from threading import Thread
import numpy as np
import struct

class AudioSignalSource(SamplerBase):
    def __init__(self, port, baud_rate=115200, batch_size=256):
        self.port = port
        self.baud_rate = baud_rate
        self.batch_size = batch_size
        self.buffer = DataBuffer()
    
    def start(self):
        self.is_active = True
        self.serial = serial.Serial(self.port, baudrate=self.baud_rate, timeout=1)
        self.thread = Thread(target=self.process_data)
        self.thread.start()

    def stop(self):
        self.is_active = False
        if self.thread is not None:
            self.thread.join()
        if self.serial is not None:
            self.serial.close()

    def process_data(self):
        while self.is_active:
            raw = self.serial.read(self.batch_size * 2)
            values =  np.asarray(struct.unpack(f"<{self.batch_size}h", raw))
            data = {"audio": values}
            self.buffer.extend(data)
        
    def read(self):
        value = self.buffer.copy()
        self.buffer.clear()
        return value


source = AudioSignalSource('/dev/tty.usbmodem11401')
source.start()

In [ ]:
source.read()

In [ ]:
source.stop()

We can also use BLE (Bluetooth Low Energy).

We could create a SignalSource from scratch where we need to deal with threading and an asynchronous BLE packet receiver.

Or we can use the BLESignalSource where we only need to create a subclass of BLEProtocol.

Our subclass needs to define an async function, data_received(self, packet), that receives a packet and transforms them into samples. 

Finally we need to put them into an async queue (defined in superclass)

The BLESignalSource should do the rest.

In [92]:
import time
import struct
import numpy as np

from genki_signals.signal_sources import BLESignalSource, BLEProtocol

ble_address = "5A37ABD2-264A-F991-6352-697F09DE0AFD" #replace with your BLE Address
char_uuid = "19b10001-e8f2-537e-4f6c-d104768a1214" #replace with your Characteristic UUID

class MyArduinoBLEProtocol(BLEProtocol):
    def __init__(self):
        super().__init__()

    # This function receives data_packets from BLE and puts them in an asyncio queue
    # No encoding
    # Single sample per packet
    async def packet_received(self, packet) -> None:
        values = struct.unpack('<11f', packet)
        data_dict = {
            "timestamp": np.array([time.time()]), 
            "acc": np.expand_dims(values[:3], axis=1),
            "gyro": np.expand_dims(values[3:6], axis=1),
            "color": np.expand_dims(values[6:10], axis=1),
            "proximity": np.array(values[10:11]),
        }
        await self.queue.put(data_dict)
        

source = BLESignalSource(ble_address, char_uuid, MyArduinoBLEProtocol)
source.start()

Connecting to device at address 5A37ABD2-264A-F991-6352-697F09DE0AFD


In [105]:
source.read()

DataBuffer(max_size=None, data=timestamp: (99,)
acc: (3, 99)
gyro: (3, 99)
color: (4, 99)
proximity: (99,))

In [104]:
source.stop()

Got a cancel message, exiting.
